In [1]:
import Random

In [2]:
include("BalanceEquation.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [3]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEquationData2D()

    endstate,steps,totaltime = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,maxsteps)
    normedvars = calcstor.statevariances / totaltime
    normedsfcovs = calcstor.statefluxcovs / totaltime
    D_ij = [calcstor.meanfluxes[1]+calcstor.meanfluxes[2] 0; 0 calcstor.meanfluxes[3]+calcstor.meanfluxes[4]]

    println("Flux Balance on Rates")
    println(relative_error(calcstor.meanfluxes[1],calcstor.meanfluxes[2]))
    println(relative_error(calcstor.meanfluxes[3],calcstor.meanfluxes[4]))

    println("Cov Balances")
    for (i,j) in [(1,1),(1,2),(2,2)]
        print(i,j)
        print(": ")
        println(balance_eq_check(normedsfcovs,D_ij,i,j))
    end

    println(steps)
    println(endstate)

    return calcstor
end

balance_test (generic function with 1 method)

In [5]:
@time calcs = balance_test([10,50],[10.,1.,4.,1.,-1.,50.],10^6,10^8,Random.MersenneTwister())

Flux Balance on Rates
0.0004855342328085931
0.00019223771586625028
Cov Balances
11: 0.5920651278998826
12: 0.5340458105070262
22: 0.3402509301520454
9989730
[7, 21]
  8.558779 seconds (109.89 M allocations: 7.294 GiB, 3.70% gc time)


BalanceEquationData2D([6.619985259020493, 26.41406070598011], [877563.4270397796 1.3585437042802935e6; 1.3585437042802935e6 9.38951175335473e6], [6.611074195491506, 6.6078658504153225, 26.43146340166129, 26.426383254105914], [-112634.24666397924 863200.9210386926 3.4528036841547703e6 1.32396741198453e6; -790283.9532550937 1.3962237066930186e6 5.584894826772074e6 9.32482476864184e6], [0.38001476947884694, -5.414061112009506], [0.4311793579717005, -0.607865896002437, -2.431463584009748, -5.426383661059443], [0.38001474097950716, -5.414060705980109])

In [68]:
function reach_balance_test(initialstate,params,numsteps,maxsteps,rng,flux_thres,cov_thres,tries)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()
    trynum = 0
    thresholdmet = false
    totalsteps = 0
    
    while (trynum < tries) && (!thresholdmet)
        new_max = trunc(Int,maxsteps*(tries-trynum)/tries)
        statevector,steps = direct_gillespie!(statevector,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,new_max)
        if (relative_error(calcstor.meanRpX,calcstor.meanRmX) < flux_thres) &&  (relative_error(calcstor.meanRpY,calcstor.meanRmY) < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum) < cov_thres
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum) < cov_thres
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum) <cov_thres 
            thresholdmet = all([cbxx_comp,cbyy_comp,cbyx_comp])               
        end
        trynum += 1
        totalsteps += steps
        print("Balance Test for chained simulation ")
        println(trynum)
        print("Current steps: ")
        println(totalsteps)
        print("Means: ")
        println([calcstor.meanx,calcstor.meany])
        print("Flux Balance on x: ")
        println(relative_error(calcstor.meanRpX,calcstor.meanRmX))
    
        print("Flux Balance on y: ")
        println(relative_error(calcstor.meanRpY,calcstor.meanRmY))
    
        print("Cov Balance on x,x: ")
        println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                                calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
        print("Cov Balance on y,y: ")
        println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                                calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
        print("Cov Balance on y,x: ")
        println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                                0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    end
    print("Tries taken: ")
    println(trynum)
    print("Total Steps: ")
    println(totalsteps)
    print("Threshold met: ")
    println(thresholdmet)
return calcstor
end

reach_balance (generic function with 1 method)

In [69]:
calcs = reach_balance_test([100,400],[10.,1.,4.,1.,-1.,50.],10^5,10^8,Random.MersenneTwister(),0.01,0.03,20)

Balance Test for chained simulation 1
Current steps: 1002014
Means: [6.647977522540511, 26.552698451158967]
Flux Balance on x: 0.005043072669317034
Flux Balance on y: 0.0008175182777894587
Cov Balance on x,x: 0.011229121212588521
Cov Balance on y,y: 0.06138630238400563
Cov Balance on y,x: 0.10288129171531248
Balance Test for chained simulation 2
Current steps: 2004566
Means: [6.634438750881015, 26.529792196278333]
Flux Balance on x: 0.0029248028932045327
Flux Balance on y: 0.0019839862167321644
Cov Balance on x,x: 0.010302289941230643
Cov Balance on y,y: 0.027404145418627616
Cov Balance on y,x: 0.07438037576969497
Balance Test for chained simulation 3
Current steps: 3008093
Means: [6.6431742187695635, 26.572020818748584]
Flux Balance on x: 0.004772571571547394
Flux Balance on y: 0.0022748585233842177
Cov Balance on x,x: 0.012120487281425788
Cov Balance on y,y: 0.01536651099384867
Cov Balance on y,x: 0.06019330212356481
Balance Test for chained simulation 4
Current steps: 4009033
Means:

BalanceEqData(6.618648083798361, 26.461924442789442, 181660.85494635877, 1.0567876898205164e6, 1.1518479075625038e7, 1.6456221917784193e6, 6.607545805198764, 6.606718096278041, 26.426872385112166, 26.4741651078378, 1.0397121113305165e6, -134863.97550653399, 1.6912737921273627e6, -958153.4985967941, 1.6037093673902424e6, 4.158848445322066e6, 1.144149324795211e7, 6.765095168509451e6)